This file implements a baseline model that is based on the assumption that if the price increases/decreases during a number of time steps, it will increase/decrease during the following number of time steps. Both of the time steps are predefined; they are the same values as input and output sequence lengths in other notebooks in this project, the notebooks with transformer or autoformer models.<br>

This baseline model works with the same target variable as other models, the closing price of a currency pair.

In [1]:
from imports import *

2024-10-23 22:22:55.941602: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-23 22:22:55.964521: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-23 22:22:55.964539: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-23 22:22:55.964557: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-23 22:22:55.969209: I tensorflow/core/platform/cpu_feature_g

In [2]:
sys.path.append('src')
from process_data import DataProcessor
from autoformer import Autoformer
from visualize_data import Visualizer
from train_model import ModelTrainer
from load_data import DataLoader
model_trainer = ModelTrainer()
data_processor = DataProcessor()
visualizer = Visualizer()
data_loader = DataLoader()

In [3]:
eurusd15m, _ = data_loader.load_eurusd_15min()
usdcad1min, _ = data_loader.load_usdcad_1min()
gbpusd5min, _ = data_loader.load_gpbusd_5min()

In [4]:
train_pct = 0.6
val_pct = 0.2

In [5]:
eurusd15m = eurusd15m[['close']]
usdcad1min = usdcad1min[['close']]
gbpusd5min = gbpusd5min[['close']]

The following function evaluates the baseline model. It receives a dataset with just a closing price, creates input and output sequences corresponding to the lengths used in other notebooks, and splits them into training, validation, and testing sets. As is done with real models, the evaluation of this one is done on the test set.

In [6]:
def determine_price_movement(data, len_in, len_out):
    Xdata, ydata = model_trainer.create_Xy_data(data, data, len_in, len_out)
    _, _, _, _, Xtest, ytest = model_trainer.split_data(Xdata, ydata, train_pct, val_pct)
    in_move = []
    out_move = []
    for i in range(0, Xtest.shape[0]):
        # for each sequence:
        # compare prices at the start and end of the input sequence and determine increase/decrease
        # compate prices at the end of the input sequence and the end of the output sequence
        # if both determinations are the same, the prediction is correct, else it is not
        first_in = Xtest[i, 0, 0]
        last_in = Xtest[i, -1, 0]
        last_out = ytest[i, -1, 0]
    
        if first_in < last_in:
            in_move.append(1)
        else:
            in_move.append(0)

        if last_in < last_out:
            out_move.append(1)
        else:
            out_move.append(0)
    print('Accuracy: ', accuracy_score(in_move, out_move))
    print('Precision:', precision_score(in_move, out_move))
    print('Recall:   ', recall_score(in_move, out_move))
    print('F1 score  ', f1_score(in_move, out_move))

## USD/CAD
Notebook: autoformer_predict_price, transformer_predict_price<br>

Both notebooks use input length 96, and output length 30.

In [7]:
input_seq_len = 96
output_seq_len = 30
determine_price_movement(usdcad1min, 96, 30)

Accuracy:  0.49383411580594677
Precision: 0.4858242463117383
Recall:    0.48150031786395425
F1 score   0.48365261813537674


## EUR/USD
Notebooks:<br>
autoformer_predict_price_move: lengths: 64-8<br>
transformer_out_seq: lengths: 64-4, 64-8, 64-16

In [8]:
input_seq_len = 64
output_seq_len = 8
determine_price_movement(eurusd15m, input_seq_len, output_seq_len)

Accuracy:  0.48939257480236165
Precision: 0.49762423282518314
Recall:    0.49492960519838536
F1 score   0.49627326126659754


In [9]:
input_seq_len = 64
output_seq_len = 4
determine_price_movement(eurusd15m, input_seq_len, output_seq_len)

Accuracy:  0.4892680242157402
Precision: 0.4976090854751943
Recall:    0.49163385826771655
F1 score   0.49460342608179025


In [10]:
input_seq_len = 64
output_seq_len = 16
determine_price_movement(eurusd15m, input_seq_len, output_seq_len)

Accuracy:  0.4865392313851081
Precision: 0.4944510503369005
Recall:    0.4916740565572963
F1 score   0.4930586433476607


## GBP/USD
Notebook:<br>
transformer_in_seq: lengths: 32-8, 16-8, 8-8

In [11]:
input_seq_len = 32
output_seq_len = 8
determine_price_movement(gbpusd5min, input_seq_len, output_seq_len)

Accuracy:  0.47335945968481613
Precision: 0.4774987358840384
Recall:    0.4683418746900314
F1 score   0.4728759806376231


In [12]:
input_seq_len = 16
output_seq_len = 8
determine_price_movement(gbpusd5min, input_seq_len, output_seq_len)

Accuracy:  0.48124374791597196
Precision: 0.47395921119164
Recall:    0.47548190733851875
F1 score   0.4747193382290875


In [13]:
input_seq_len = 8
output_seq_len = 8
determine_price_movement(gbpusd5min, input_seq_len, output_seq_len)

Accuracy:  0.48670500958572976
Precision: 0.4832293949098264
Recall:    0.48112099345527776
F1 score   0.4821728893373697
